In [83]:
import pandas as pd 
import jieba
from collections import Counter
import numpy as np

In [1]:
data= open('./toutiao_cat_data.txt').readlines()
data[0]

'6551700932705387022_!_101_!_news_culture_!_京城最值得你来场文化之旅的博物馆_!_保利集团,马未都,中国科学技术馆,博物馆,新中国\n'

In [14]:
data = [d.split('_!_') for d in data]

AttributeError: 'list' object has no attribute 'split'

In [57]:
df = pd.DataFrame(data, columns=['id', 'code', 'category', 'content', 'keys'])
df = df[:50000]
NUM_DOCUMENTS = len(df)
df.head()

,id,code,category,content,keys
0,6551700932705387022,101,news_culture,京城最值得你来场文化之旅的博物馆,"保利集团,马未都,中国科学技术馆,博物馆,新中国\n"
1,6552368441838272771,101,news_culture,发酵床的垫料种类有哪些？哪种更好？,\n
2,6552407965343678723,101,news_culture,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,\n
3,6552332417753940238,101,news_culture,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,\n
4,6552475601595269390,101,news_culture,黄杨木是什么树？,\n


In [12]:
string = '京城最值得你来场文化之旅的博物馆'
words = list(jieba.cut(string))
' '.join(words)

'京城 最 值得 你 来场 文化 之旅 的 博物馆'

In [ ]:
def foo(x):
    x = jieba.cut(x)
    x = ' '.join(x)
    reutrn x
    
# 
lambda x: ' '.join(jieba.cut(x))

In [69]:
df['content'] = df.content.apply(lambda x: ' '.join(jieba.cut(x)))

In [42]:
df.head()

,id,code,category,content,keys
0,6551700932705387022,101,news_culture,京城 最 值得 你 来场 文化 之旅 的 博物馆,"保利集团,马未都,中国科学技术馆,博物馆,新中国\n"
1,6552368441838272771,101,news_culture,发酵 床 的 垫料 种类 有 哪些 ？ 哪 种 更好 ？,\n
2,6552407965343678723,101,news_culture,上联 ： 黄山 黄河 黄皮肤 黄土高原 。 怎么 对 下联 ？,\n
3,6552332417753940238,101,news_culture,林徽因 什么 理由 拒绝 了 徐志摩 而 选择 梁思成 为 终身伴侣 ？,\n
4,6552475601595269390,101,news_culture,黄杨木 是 什么 树 ？,\n


In [38]:
# 创建字典
vocab = []
for document in df.content:
    words = document.split()
    vocab.extend(words) # +就可以了吧 YEs

# 总单词的数量
NUM_TERMS = len(vocab)

# 词汇表
# 单词根据词频排序
vocab = dict(Counter(vocab))
vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
vocab = dict(vocab)

# 过滤
# 词频少于100词的单词

vocab = {k:v for k,v in vocab.items() if v>100}
# stopwords = ['了','的']
# vocab = {k:v for k,v in vocab.items() if k not in stopwords}

# 单词2id
# id2单词
word2id = {k:v for v,k in enumerate(vocab.keys())}
id2word = {v:k for k,v in word2id.items()}

NUM_WORDS = len(vocab)
NUM_WORDS,NUM_TERMS

(693, 658540)

# 创建索引

``` 
way1

         文档1  文档2 ...
关键字1    1      0
关键字2    0      1
....
```

In [47]:
def generate_index(x):
    """
    将字符串转换成 id list
    """
    
    index = [0] * NUM_WORDS
    for word in x.split():
        i = word2id.get(word, -1)
        if i == -1: continue
        index[i] = 1
        
    return index

# 
# generate_index('京城 最 值得 你 来场 文化 之旅 的 博物馆')

In [48]:
# 生成索引大表 以后进行使用
all_index = df.content.apply(generate_index)

In [ ]:
关键字 文档1 文档2

In [55]:
all_index = np.array(all_index.tolist()).T
all_index.shape

(693, 50000)

索引表
检索
'北京'
row = word2id['北京']
000110001010....
row2 = word2id['房价']
000010110001010...

000010101010010

逐位and
00010001


北京 010
房价 011
多少 111
     010 and 011 and 111
     0 and 0 and 1 _
     1 and 1 and 1 _
     0 and 1 and 1 _

In [56]:
query = '北京的房价多少'

query = list(jieba.cut(query))
query

['北京', '的', '房价', '多少']

计算每个关键字的TF

In [ ]:
639 * 500000

In [62]:
def querybykeys(query):
    """
    查找关键字都出现的文档
    """
    finds = np.array([1] * NUM_DOCUMENTS)
    
    for key in query:
        finds += all_index[word2id[key]]
        finds = finds == 2
        finds = finds.astype(int)
    return finds

finds = querybykeys(query)


Find document id that contains all keys: []


In [66]:
# 既然没有那咱找几个小的数据进行继续的说明
finds = querybykeys(['北京', '房价']) # ['北京', '房价']
finds = [i for i,j in enumerate(finds) if j]
print('Find document id that contains all keys:', finds)

# 将这些数据打印出来
for d in finds:
    print(df.iloc[d].content)

Find document id that contains all keys: [11848, 21857, 23181, 23516, 25454, 28720, 29154, 45713]
房价白跌了！北京首套房贷款利率上浮10%
你认为北京未来5年的房价会下跌吗？
「重磅」北京限房价项目销售办法出炉 部分将收作共有产权房
北京的房价还会继续上涨吗？
北京房贷利率下周再涨 专家：累计相当于房价上调了10%
北京限价房转共有产权房政策拟出炉 对房价有何影响？
房价不涨卖不动，北京首套房利率继续上浮，刚需招惹了谁？
北京的房价还会继续上涨吗？


In [79]:
# 计算每个数据的TF

tf_result = []
for d in finds:
    words = df.iloc[d].content.split()
    print(words)
    # 统计单词的出现次数
    tf = dict(Counter(words))
    num_words = len(words)
    # 统计词频
    # 需要注意一下 
    # 手稿里 v/len(tf) 是不对的 
    tf = {k:round(v/num_words, 4) for k,v in tf.items() if k in query}
    print()
    print(words)
    print(tf)
    print(sum(tf.values()))
    tf_result.append(sum(tf.values()))

i = np.argmax(tf_result) # [scoer1, score2,... ] max
print(f'tf:{max(tf_result)} 检索结果:{df.iloc[finds[i]].content}')

['房价', '白跌', '了', '！', '北京', '首', '套房', '贷款', '利率', '上浮', '10%']

['房价', '白跌', '了', '！', '北京', '首', '套房', '贷款', '利率', '上浮', '10%']
{'房价': 0.0909, '北京': 0.0909}
0.1818
['你', '认为', '北京', '未来', '5', '年', '的', '房价', '会', '下跌', '吗', '？']

['你', '认为', '北京', '未来', '5', '年', '的', '房价', '会', '下跌', '吗', '？']
{'北京': 0.0833, '的': 0.0833, '房价': 0.0833}
0.2499
['「', '重磅', '」', '北京', '限', '房价', '项目', '销售', '办法', '出炉', '部分', '将', '收作', '共有', '产权', '房']

['「', '重磅', '」', '北京', '限', '房价', '项目', '销售', '办法', '出炉', '部分', '将', '收作', '共有', '产权', '房']
{'北京': 0.0625, '房价': 0.0625}
0.125
['北京', '的', '房价', '还会', '继续', '上涨', '吗', '？']

['北京', '的', '房价', '还会', '继续', '上涨', '吗', '？']
{'北京': 0.125, '的': 0.125, '房价': 0.125}
0.375
['北京', '房贷利率', '下周', '再涨', '专家', '：', '累计', '相当于', '房价', '上调', '了', '10%']

['北京', '房贷利率', '下周', '再涨', '专家', '：', '累计', '相当于', '房价', '上调', '了', '10%']
{'北京': 0.0833, '房价': 0.0833}
0.1666
['北京', '限价', '房转', '共有', '产权', '房', '政策', '拟', '出炉', '对', '房价', '有何', '影响', '？']

['北京', '限价', '房转', '共有',

In [82]:
corpus = []
for document in df.content:
    corpus.append(document.split())

In [84]:
# IDF

idf = {}
for word in vocab:
    count = 0
    for line in corpus:
        if word in line:
            count += 1
    
    idf[word] = round(np.log(NUM_DOCUMENTS / count), 4)
    
idf

{'，': 0.6122,
 '？': 0.5814,
 '的': 1.0274,
 '！': 1.9467,
 '：': 2.0292,
 '了': 2.1684,
 '是': 2.2236,
 '你': 2.2624,
 '吗': 2.4169,
 '有': 2.4525,
 '“': 2.566,
 '”': 2.5689,
 '在': 2.5505,
 '什么': 2.6558,
 '怎么': 2.6627,
 '如何': 2.7436,
 '为什么': 2.7405,
 '和': 2.9866,
 '中国': 3.0437,
 '被': 3.007,
 '都': 3.0295,
 '人': 3.094,
 '看': 3.0696,
 '会': 3.073,
 '不': 3.1217,
 '对': 3.2294,
 '》': 3.3055,
 '《': 3.3066,
 '上': 3.3507,
 '、': 3.744,
 '美国': 3.4321,
 '这': 3.3885,
 '如果': 3.4076,
 '最': 3.5059,
 '好': 3.5706,
 '能': 3.5336,
 '年': 3.6082,
 '上联': 3.5614,
 '下联': 3.5806,
 '要': 3.6238,
 '哪些': 3.6605,
 '一个': 3.7132,
 '还': 3.6985,
 '大': 3.7448,
 '就': 3.774,
 '后': 3.8059,
 '我': 3.8661,
 '买': 3.9001,
 '农村': 3.8158,
 '到': 3.8149,
 '说': 3.8444,
 '世界': 3.869,
 '谁': 3.869,
 '5': 3.914,
 '他': 3.8991,
 '中': 3.8642,
 '去': 3.8971,
 '—': 4.5375,
 '可以': 3.917,
 '多': 3.9373,
 '现在': 3.9373,
 '让': 3.9581,
 '万': 4.0444,
 '网友': 3.9591,
 '2018': 3.9665,
 '3': 3.9889,
 '与': 4.0052,
 '日本': 4.1851,
 '却': 4.0241,
 '没有': 4.0536,
 '还是': 4

In [87]:
sorted(idf.items(), key=lambda x:x[1], reverse=True)
pass

In [88]:
print('索引找到的数据子集：')
for d in finds:
    print(df.iloc[d].content)
print()

索引找到的数据子集：
房价 白跌 了 ！ 北京 首 套房 贷款 利率 上浮 10%
你 认为 北京 未来 5 年 的 房价 会 下跌 吗 ？
「 重磅 」 北京 限 房价 项目 销售 办法 出炉   部分 将 收作 共有 产权 房
北京 的 房价 还会 继续 上涨 吗 ？
北京 房贷利率 下周 再涨   专家 ： 累计 相当于 房价 上调 了 10%
北京 限价 房转 共有 产权 房 政策 拟 出炉   对 房价 有何 影响 ？
房价 不涨 卖 不动 ， 北京 首 套房 利率 继续 上浮 ， 刚需 招惹 了 谁 ？
北京 的 房价 还会 继续 上涨 吗 ？



In [92]:
tfidf_result = []
for d in finds:
    words = df.iloc[d].content.split()
    tf = dict(Counter(words))
    # 统计词频
    tf = {k:round(v/len(words),4) for k,v in tf.items() if k in query}
    #计算TFIDF
    tfidf = [v*idf[k]  for k,v in tf.items()]
    [print(f'word:{k} tf:{v} idf:{idf[k]}') for k,v in tf.items()]
    print()
    tfidf_result.append(sum(tfidf))
    
    
index = np.argmax(tfidf_result)
row_id = finds[index]
print(tfidf_result)
print(f'tfidf:{max(tfidf_result)}, data:{df.iloc[row_id].content}')

word:房价 tf:0.0909 idf:4.3917
word:北京 tf:0.0909 idf:5.1568

word:北京 tf:0.0833 idf:5.1568
word:的 tf:0.0833 idf:1.0274
word:房价 tf:0.0833 idf:4.3917

word:北京 tf:0.0625 idf:5.1568
word:房价 tf:0.0625 idf:4.3917

word:北京 tf:0.125 idf:5.1568
word:的 tf:0.125 idf:1.0274
word:房价 tf:0.125 idf:4.3917

word:北京 tf:0.0833 idf:5.1568
word:房价 tf:0.0833 idf:4.3917

word:北京 tf:0.0714 idf:5.1568
word:房价 tf:0.0714 idf:4.3917

word:房价 tf:0.0588 idf:4.3917
word:北京 tf:0.0588 idf:5.1568

word:北京 tf:0.125 idf:5.1568
word:的 tf:0.125 idf:1.0274
word:房价 tf:0.125 idf:4.3917

[0.8679586499999998, 0.88097247, 0.59678125, 1.3219875, 0.79539005, 0.6817629000000001, 0.5614518, 1.3219875]
tfidf:1.3219875, data:北京 的 房价 还会 继续 上涨 吗 ？


In [ ]:
tfidf 作为额外的特征 加入到以后的数据特征中